# 07wk-1: (합성곱신경망) – CNN 자랑, CNN 핵심레이어

## 1. Imports

In [1]:
import torch
import matplotlib.pyplot as plt
import torchvision

In [2]:
plt.rcParams['figure.figsize'] = (4.5, 3.0)

## 2. 주요 코드 등

```Python
net = torch.nn.Sequential(
    ## 1 layer - 이미지를 펴지 않고도 할 수 있음
    torch.nn.Conv2d(in_channels = 1, out_channels = 64, kernel_size = 5, stride = 1), ## 1은 흑백, 3은 컬러, kernel_size는 작게 잡을수록 세세하게 봄
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size = 3), ## 특징을 추림, 작게 잡을수록 원본을 유지함

    ## transform
    torch.nn.Flatten(),
    torch.nn.Linear(???, p) ## Flatten까지만 살펴보고 직접 집어넣어야 함 -> 그냥 클래스 하나 만들어서 shape 찍어도 됨
).to("cuda:0")

loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(net.parameters())

##---##
for epoc in range(1, 200) :
    """epoches"""
```

## 3. CNN 자랑

* SoftMax : 다항분류에서 로짓이 여러 개 나왔을 때 취하는 것. 지수함수 먹이고 상대비율로 확률 추정
> 세 개 이상의 분류별 추정 로짓 -> 지수 먹임 -> 전체합으로 나눔

* Sigmoid : 한개의 로짓
> 한 개의 로짓과 0을 비교 -> 오즈, 1 -> 1일 확률 / 0일 확률로 분리

### A. 성능 좋음

In [3]:
train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True)
train_dataset = torch.utils.data.Subset(train_dataset, range(5000))
test_dataset = torch.utils.data.Subset(test_dataset, range(1000))
to_tensor = torchvision.transforms.ToTensor()
X = torch.stack([to_tensor(img) for img, lbl in train_dataset]).to("cuda:0")
y = torch.tensor([lbl for img, lbl in train_dataset])
y = torch.nn.functional.one_hot(y).float().to("cuda:0")
XX = torch.stack([to_tensor(img) for img, lbl in test_dataset]).to("cuda:0")
yy = torch.tensor([lbl for img, lbl in test_dataset])
yy = torch.nn.functional.one_hot(yy).float().to("cuda:0")

In [4]:
X.shape ## 너무 많아서 좀 자름

torch.Size([5000, 1, 28, 28])

*개억지 발악딜 신경망*

In [7]:
net = torch.nn.Sequential(
    torch.nn.Flatten(), ## 다차원 데이터 펴버림
    torch.nn.Linear(784,2048),
    torch.nn.ReLU(),
    torch.nn.Linear(2048,10)
).to("cuda")
loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(net.parameters())

for epoc in range(1,500):
    #1
    logits = net(X)
    #2
    loss = loss_fn(logits, y) 
    #3
    loss.backward()
    #4 
    optimizr.step()
    optimizr.zero_grad()

In [8]:
(net(X).argmax(axis=1) == y.argmax(axis=1)).float().mean()

tensor(1., device='cuda:0')

In [9]:
(net(XX).argmax(axis=1) == yy.argmax(axis=1)).float().mean()

tensor(0.8460, device='cuda:0')

***대충 설계한 합성곱***

In [11]:
net = torch.nn.Sequential(
    torch.nn.Conv2d(1,16,2),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(2),
    torch.nn.Flatten(),
    torch.nn.Linear(2704,10), ## 왜 이렇게 됨???
).to("cuda")
loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(net.parameters())

for epoc in range(1,500):
    #1
    logits = net(X)
    #2
    loss = loss_fn(logits, y) 
    #3
    loss.backward()
    #4 
    optimizr.step()
    optimizr.zero_grad()

In [12]:
(net(X).argmax(axis=1) == y.argmax(axis=1)).float().mean()

tensor(0.9646, device='cuda:0')

In [13]:
(net(XX).argmax(axis=1) == yy.argmax(axis=1)).float().mean()

tensor(0.8610, device='cuda:0')

> 대충해도 더 잘됨

In [16]:
net[:-1](X).shape  ## 마지막 호출해서 직접 차원을 알아야 함

torch.Size([5000, 2704])

### B. 파라메터 적음

In [20]:
## 네트워크는 분할해서 올릴 수 없으므로 ㅈㄴ 비쌈
net1 = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784,2048), ## 784*2048 + 2048
    torch.nn.ReLU(),
    torch.nn.Linear(2048,10)   ## 2048*10 + 2048
)

## 파라미터 수가 훨씬 적은데도 성능도 좋음 -> 개사기
net2 = torch.nn.Sequential(
    torch.nn.Conv2d(1,16,2),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(2),
    torch.nn.Flatten(),
    torch.nn.Linear(2704,10),
)

In [26]:
net1_params = list(net1.parameters())
print(net1_params[0].shape) ## x에 대한 파라메터
print(net1_params[1].shape) ## Bias
print(net1_params[2].shape) ## x에 대한 파라메터 2
print(net1_params[3].shape) ## Bias 2

torch.Size([2048, 784])
torch.Size([2048])
torch.Size([10, 2048])
torch.Size([10])


In [28]:
2048*784 + 2048 + 10*2048 + 10

1628170

In [27]:
net2_params = list(net2.parameters())
print(net2_params[0].shape) ## x에 대한 파라메터
print(net2_params[1].shape) ## Bias
print(net2_params[2].shape) ## x에 대한 파라메터 2
print(net2_params[3].shape) ## Bias 2

torch.Size([16, 1, 2, 2])
torch.Size([16])
torch.Size([10, 2704])
torch.Size([10])


In [29]:
16*1*2*2 + 16 + 10*2704 + 10

27130

> 파라메터를 0.016만큼만 썼음에도 더 성능이 좋음, 대충대충 만들었음에도 성능이 좋음

### C. 유명함

`-` <https://brunch.co.kr/@hvnpoet/109>

* 딥러닝 슈퍼스타
> 힌튼, 르쿤, 벤지오, 응
>
> 힌튼 교수넴 -> DBN(사이언스지) -> 깊은신경망을 만들어도 학습할 수 있다, 성능도 좋다.
>
> 관심 없음 --- 휘하 대학원생 알렉스가 CIFAR10 데이터셋을 만들어서 분류 문제 증명하려고 함 -> 힘듦
>
> 다른대학원생 -> 콘테스트 나가서 입증해보자 -> 컴퓨터 느림, 오버피팅 생김, 국지적 최소값, 기울기 소멸
>
> ---> GPU 쓰자, 드랍아웃 쓰자, 렐루를 활성화함수로 쓰자 ---> ㅈㄴ 잘함, 혁신

## 4. CNN 핵심레이어

```Python
torch.nn.Conv2d(),
torch.nn.ReLU(),
torch.nn.MaxPool2d()
```

### A. `torch.nn.ReLU`

**(예시1) 연산방법, kernel_size의 의미**

In [30]:
img = torch.randn(1, 1, 4, 4) ## (4, 4) 흑백 이미지 한장
relu = torch.nn.ReLU()

In [31]:
img

tensor([[[[ 1.4084,  1.1233,  0.7891, -2.4956],
          [ 0.0470, -1.2522, -0.6157, -0.4172],
          [ 0.1714,  0.5501,  0.9884, -0.9269],
          [ 0.7274, -0.8939,  0.9855, -0.8834]]]])

In [32]:
relu(img)

tensor([[[[1.4084, 1.1233, 0.7891, 0.0000],
          [0.0470, 0.0000, 0.0000, 0.0000],
          [0.1714, 0.5501, 0.9884, 0.0000],
          [0.7274, 0.0000, 0.9855, 0.0000]]]])

> 그냥 똑같음. 음수이면 0으로 올림

In [33]:
img = torch.randn(1, 1, 4, 4)
mp = torch.nn.MaxPool2d(kernel_size = 2)

In [35]:
img

tensor([[[[ 0.3012,  0.2972,  0.1387, -0.1596],
          [ 0.6020,  0.3204,  0.0589, -1.3579],
          [ 0.4615, -1.1834, -1.0951, -0.8815],
          [ 0.5930,  0.3248,  0.4900, -0.8284]]]])

In [34]:
mp(img)

tensor([[[[0.6020, 0.1387],
          [0.5930, 0.4900]]]])

> (4, 4)의 이미지를 크기가 2인 정방 윈도우로 쪼개어 각 윈도우의 Max값을 추출함

**(예시2) 이미지크기와 딱 맞지 않는 커널일 경우?**

In [39]:
img = torch.randn(1, 1, 5, 5)
mp = torch.nn.MaxPool2d(kernel_size = 2)

In [40]:
img

tensor([[[[-0.7601, -0.3066, -0.5217, -0.6908, -0.5589],
          [-0.7216,  0.4627,  0.1489, -0.2923, -0.6556],
          [ 0.5201, -0.5569, -0.9285, -1.2374,  0.0843],
          [-1.0531, -0.8987, -0.5979,  1.1153, -0.3969],
          [-1.7311, -0.4479,  0.5167, -0.7048,  1.3095]]]])

In [41]:
mp(img)

tensor([[[[0.4627, 0.1489],
          [0.5201, 1.1153]]]])

> **파이토치** : 남는 차원은 그냥 가차없이 버림

**(예시3) 정사각형이 아닌 커널** <- 잘 안씀

In [42]:
img = torch.rand(1, 1, 4, 4)
mp = torch.nn.MaxPool2d(kernel_size = (2, 4))

In [43]:
img

tensor([[[[0.0845, 0.4076, 0.0115, 0.3949],
          [0.1350, 0.9975, 0.1458, 0.1850],
          [0.0292, 0.5169, 0.6951, 0.8657],
          [0.1442, 0.3767, 0.5413, 0.3423]]]])

In [44]:
mp(img)

tensor([[[[0.9975],
          [0.8657]]]])

### C. `torch.nn.Conv2d`

**(예시1) 연산방법, stride = 2**

In [45]:
img = torch.rand(1, 1, 4, 4) ## input channel : 1과 3. 이미지 형태
conv = torch.nn.Conv2d(in_channels = 1, out_channels = 1, kernel_size = 2, stride = 2)

In [46]:
img

tensor([[[[0.1054, 0.0379, 0.1375, 0.0127],
          [0.6344, 0.0410, 0.4574, 0.8073],
          [0.1375, 0.3647, 0.3059, 0.3496],
          [0.7166, 0.1335, 0.2234, 0.9699]]]])

In [47]:
conv(img)

tensor([[[[-0.5347, -0.7454],
          [-0.5788, -0.7303]]]], grad_fn=<ConvolutionBackward0>)

???

`-` 도출과정

In [49]:
img[:, :, :2, :2]

tensor([[[[0.1054, 0.0379],
          [0.6344, 0.0410]]]])

In [50]:
img[:, :, :2, 2:]

tensor([[[[0.1375, 0.0127],
          [0.4574, 0.8073]]]])

In [51]:
img[:, :, 2:, :2]

tensor([[[[0.1375, 0.3647],
          [0.7166, 0.1335]]]])

In [52]:
img[:, :, 2:, 2:]

tensor([[[[0.3059, 0.3496],
          [0.2234, 0.9699]]]])

In [54]:
conv.weight, conv.bias

(Parameter containing:
 tensor([[[[ 0.1550, -0.0092],
           [-0.1932, -0.3263]]]], requires_grad=True),
 Parameter containing:
 tensor([-0.4148], requires_grad=True))

> 파라미터가 있음

In [56]:
(img[:, :, :2, :2]*conv.weight.data).sum() + conv.bias.data ## 첫 번째 원소 나옴

tensor([-0.5347])

In [57]:
(img[:, :, :2, 2:]*conv.weight.data).sum() + conv.bias.data ## (1, 2) 원소

tensor([-0.7454])

In [58]:
(img[:, :, 2:, :2]*conv.weight.data).sum() + conv.bias.data ## (2, 1) 원소

tensor([-0.5788])

In [59]:
(img[:, :, 2:, 2:]*conv.weight.data).sum() + conv.bias.data ## (2, 2) 원소

tensor([-0.7303])

> `conv.weight.data`의 값과 동일한 위치의 값을 곱하여(행렬곱 아님) 싹 더함 --> 가중평균 느낌
>
> 거기다가 bias 더해줌
>
> 약간 스무딩하는 느낌으로다가 생각하면 될듯